In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose  
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.rcParams.update({'figure.figsize':(16,3), 'figure.dpi':100})

In [ ]:
# Leitura do arquivo Excel
series = pd.read_excel('spare-parts-sales.xlsx', header=0, 
                       names=['item', 'sales', 'cost', 'date'], index_col=None, parse_dates=True, squeeze=True)

# Filtra o arquivo para os anos de 2014 a 2016
series['date'] = pd.to_datetime(series['date'])
date_filter = (series['date'] >= '2014-01-01') & (series['date'] <= '2016-11-30')
series = series[date_filter]

# Filtra para o item '98550154' (óleo de motor)
sku = '98550154'
series = series[series.item == sku]

# Exclui a coluna item e cost
series.drop(series.columns[[0, 2]], axis=1, inplace=True)

# Transforma para vendas semanais
series = (series.groupby(pd.Grouper(key='date',freq='W')).sum()).reset_index()

In [ ]:
# Função para cálculo do teste de Dickey-Fuller
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    print('Null Hypothesis: Unit Root Present')
    print('Test Statistic < Critical Value => Reject Null')
    print('P-Value =< Alpha(.05) => Reject Null\n')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput[f'Critical Value {key}'] = value
    print (dfoutput, '\n')
    
# Função para autocorrelação e autocorrelação parcial
def plot_acf_pacf(timeseries, lags):
    fig, axes = plt.subplots(1,2,figsize=(16,3), dpi= 100)
    plot_acf(timeseries.tolist(), lags=lags, ax=axes[0])
    plot_pacf(timeseries.tolist(), lags=lags, ax=axes[1])

In [ ]:
media_movel = series['sales'].rolling(window=4).mean() # Cálculo da Média Móvel
desvio_movel = series['sales'].rolling(window=4).std() # Cálculo do Desvio Móvel
orig = plt.plot(series['sales'], color='blue', label='Observado') # Gráfico com Valores Originais
media = plt.plot(media_movel, color='red', label='Média Móvel') # Gráfico da Média Móvel
desvio = plt.plot(desvio_movel, color='black', label='Desvio Padrão Móvel') # Gráfico do Desvio Móvel
plt.legend(loc='best')
plt.title('Estatísticas de rolagem')
plt.show(block=False)

In [ ]:
# Chamada para o teste de Dickey-Fuller
adf_test(series['sales'])

In [ ]:
# Gráfico de autocorrelação e autocorrelação parcial
plot_acf_pacf(series['sales'], 16)

In [ ]:
# Diferenciação para verificar se a série será estacionária
series_diff = (series['sales'] - series['sales'].shift(1)).dropna()

In [ ]:
media_movel = series_diff.rolling(window=4).mean() # Cálculo da Média Móvel
desvio_movel = series_diff.rolling(window=4).std() # Cálculo do Desvio Móvel
orig = plt.plot(series_diff, color='blue', label='Diferenciação') # Gráfico com Valores Diferenciados
media = plt.plot(media_movel, color='red', label='Média Móvel') # Gráfico da Média Móvel
desvio = plt.plot(desvio_movel, color='black', label='Desvio Padrão Móvel') # Gráfico do Desvio Móvel
plt.legend(loc='best')
plt.title('Estatísticas de rolagem')
plt.show(block=False)

In [ ]:
# Chamada para o teste de Dickey-Fuller
adf_test(series_diff)

In [ ]:
plt.rc("figure", figsize=(16,8))
result = seasonal_decompose(series_diff, period=1)  
result.plot()  
plt.show()